In [1]:
%load_ext autoreload
%autoreload 2

import pysam

import re
import os
import sys
import itertools
sys.path.append('/home/users/pjh/scripts/python_genome_package_dev/')
# from handygenome import common
# from handygenome.common import ChromDict

# FASTA_PATH = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
# FASTA_PATH_38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

# chromdict = ChromDict(fasta_path = FASTA_PATH)
# chromdict_38 = ChromDict(fasta_path = FASTA_PATH_38)
# fasta = pysam.FastaFile(FASTA_PATH)
# fasta_38 = pysam.FastaFile(FASTA_PATH_38)

In [2]:
from handygenome.igvhandle import IGVHandle
igv = IGVHandle(60387)

# Scratches

In [4]:
vcf_path = '/home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.snps.vcf.gz'
vcf = pysam.VariantFile(vcf_path)
vr = next(vcf.fetch())

[W::hts_idx_load3] The index file is older than the data file: /home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.snps.vcf.gz.tbi


In [28]:
print(vr)

1	10039	.	A	G	.	str10	DP=95;SS=1;SSC=3;GPV=0.029575;SPV=0.40784	GT:GQ:DP:RD:AD:FREQ:DP4	0/1:.:52:49:2:3.92%:32,17,2,0	0/1:.:43:39:3:7.14%:30,9,3,0



In [29]:
new_vr = vr.copy()

In [32]:
del new_vr.info['DP']

In [33]:
print(new_vr)

1	10039	.	A	G	.	str10	SS=1;SSC=3;GPV=0.029575;SPV=0.40784	GT:GQ:DP:RD:AD:FREQ:DP4	0/1:.:52:49:2:3.92%:32,17,2,0	0/1:.:43:39:3:7.14%:30,9,3,0



In [34]:
hdr = vr.header.copy()

In [35]:
hdr.info.remove_header('DP')

In [36]:
out_vcf_path = '/home/users/pjh/tmp/zzz.vcf.gz'
with pysam.VariantFile(out_vcf_path, mode='wz', header=hdr) as out_vcf:
    out_vcf.write(new_vr)

In [37]:
new_vcf = pysam.VariantFile(out_vcf_path)
new_vr = next(new_vcf.fetch())

[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/tmp/zzz.vcf.gz'


In [38]:
print(new_vr)

1	10039	.	A	G	.	str10	SS=1;SSC=3;GPV=0.029575;SPV=0.40784	GT:GQ:DP:RD:AD:FREQ:DP4	0/1:.:52:49:2:3.92%:32,17,2,0	0/1:.:43:39:3:7.14%:30,9,3,0



# Varscan output

In [2]:
snp_vcf_path = '/home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.snps.vcf.gz'
indel_vcf_path =  '/home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.indels.vcf.gz'
snp_vcf = pysam.VariantFile(snp_vcf_path)
indel_vcf = pysam.VariantFile(indel_vcf_path)

[W::hts_idx_load3] The index file is older than the data file: /home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.snps.vcf.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: /home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.indels.vcf.gz.tbi


In [169]:
bam_path_dict = {
    'tumor': '/home/users/jinsil/project/01_MUTYH/03_EGA/02_BAM/dna_bam/PD44889/PD44889a_lo0002.md.bam',
    'normal': '/home/users/jinsil/project/01_MUTYH/03_EGA/02_BAM/dna_bam/PD44889/PD44889e.md.bam',
}
bam_dict = {sid: pysam.AlignmentFile(f) for sid, f in bam_path_dict.items()}

In [104]:
igv.load(list(bam_path_dict.values()))

OK
OK


In [124]:
tmp = set()
# for vr in itertools.chain(snp_vcf.fetch(), indel_vcf.fetch()):
for vr in snp_vcf.fetch():
    REFsp = vr.ref.split('/')
    ALTsp = vr.alts[0].split('/')
    if len(REFsp) > 1:
        print(vr)
    # tmp.add(len(vr.alts[0].split('/')))
    # tmp.add(len(vr.ref.split('/')))
    # mat = re.fullmatch(r'([ACGTN]+)/(-)?([ACGTN]+)', vr.alts[0])
    # if mat is not None:
    #     len_g1 = len(mat.group(1))
    #     len_g3 = len(mat.group(3))
    #     if len_g1 > 1 a
    #     print(len(mat.group(1)), len(mat.group(3)))
    #     break

In [8]:
print(vr)

1	51864	.	C	CA/-A	.	str10	DP=46;SS=5;SSC=0;GPV=1;SPV=0.61756	GT:GQ:DP:RD:AD:FREQ:DP4	0/1:.:31:27:4:12.9%:16,11,4,0	0/1:.:15:12:2:14.29%:6,6,2,0



In [155]:
def gen():
    for vr in indel_vcf.fetch():
        REFsp = vr.ref.split('/')
        ALTsp = vr.alts[0].split('/')
        # if len(REFsp) > 1:
        if len(REFsp) > 1 and set(vr.filter) == {'PASS'} and vr.info['SOMATIC']:
        # if set(vr.filter) == {'PASS'} and vr.info['SOMATIC']:
            yield vr
        
multi_gen = gen()

In [156]:
vr = next(multi_gen)
print(vr)

StopIteration: 

In [157]:
import handygenome.vcfeditor.varscan_editing as varscan_editing
vcfspec = varscan_editing.extract_vcfspec(vr, 'hg19')
print(vcfspec)
print(vr)

Vcfspec(chrom='1', pos=10,058, ref='CCCT', alts=('C',))
1	10058	.	CCCT	C	.	PASS	DP=127;SOMATIC;SS=2;SSC=6;GPV=1;SPV=0.20594	GT:GQ:DP:RD:AD:FREQ:DP4	0/0:.:68:68:0:0%:45,23,0,0	0/1:.:59:55:2:3.51%:39,16,1,1



In [158]:
import handygenome.vcfeditor.initvcf as initvcf
import handygenome.common as common

header = initvcf.create_header(common.DEFAULT_CHROMDICTS['hg19'])
new_vr = header.new_record()
vcfspec.apply_to_vr(new_vr)
print(new_vr)
# vcfspec

1	10058	.	CCCT	C	.	.	.



In [159]:
# infile_path = snp_vcf_path
# outfile_path = '/home/users/pjh/tmp/varscan_edited_snv.vcf.gz'

varscan_editing.modify_varscan_vcf(
    snp_vcf_path, 
    '/home/users/pjh/tmp/varscan_edited_snv.vcf.gz', 
    refver='hg19',
)

varscan_editing.modify_varscan_vcf(
    indel_vcf_path, 
    '/home/users/pjh/tmp/varscan_edited_indel.vcf.gz', 
    refver='hg19',
)

[W::hts_idx_load3] The index file is older than the data file: /home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.snps.vcf.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: /home/users/jinsil/project/01_MUTYH/03_EGA/03_vcf/03_varscan2/PD44889a_lo0002.varscan.indels.vcf.gz.tbi


In [160]:
from handygenome.variant.variantplus import VariantPlusList
vplist = VariantPlusList.from_vcf('/home/users/pjh/tmp/varscan_edited_indel.vcf.gz')

[W::hts_idx_load3] The index file is older than the data file: /home/users/pjh/scripts/python_genome_packages/data/popfreq/dbSNP_b155_GRCh37.p13.vcf.gz.csi
[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/tmp/varscan_edited_indel.vcf.gz'
[KST 2022-11-29 22:23:53.971] VariantPlusList: Processing 10,000th line
[KST 2022-11-29 22:23:55.209] VariantPlusList: Processing 20,000th line
[KST 2022-11-29 22:23:56.367] VariantPlusList: Processing 30,000th line
[KST 2022-11-29 22:23:57.546] VariantPlusList: Processing 40,000th line
[KST 2022-11-29 22:23:58.751] VariantPlusList: Processing 50,000th line
[KST 2022-11-29 22:23:59.985] VariantPlusList: Processing 60,000th line


In [205]:
vp = vplist.sample()

In [206]:
vp

<VariantPlus(vcfspec=Vcfspec(chrom='2', pos=80,315,000, ref='CTT', alts=('C',)), gene=(alt_index 0: ))>

In [207]:
vp.show_igv(igv, bam_dict)

OK
OK
OK
OK
OK
OK
OK


In [166]:
igv.goto([vp.vcfspec], width=100)
igv.cmd('collapse')

OK
OK


# Read

In [3]:
bam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.tumor.bam'

In [4]:
bam = pysam.AlignmentFile(bam_path)

In [5]:
read = next(bam.fetch())

In [6]:
print(read.to_string())

ST-E00130:582:HC7KJALXX:1:2204:12307:39616	129	1	9999	0	46S66M39S	5	18606670	0	AAGTATGAGTTGCCTAGAACAGCTCTTCCTTTATTTTCTTTTTCTGGATAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACAGAGCTAACTATTGATACAACACCTTCATGACCCTACGG	<<<<<B<<<<<<<<<<<<<<<<<<<<<<<<<<<7<<<7B<7<<<BB<7<<<'<0BB<<<<B0<<7<<0<77<B<<0'<7<<7<7BB<BB<B<B<<BB7B<BB<<BB<<B<7BB'07'<<0'BBBB<B<<<7B7B<<777B<B0<<0<<'''	SA:Z:5,18606943,-,96S55M,0,0;5,18606895,-,41M110S,0,2;	MC:Z:69M2D38M44S	MD:Z:66	NM:i:0	MQ:i:60	AS:i:66	XS:i:65	RG:Z:JKL-LU-14-cancer-wgs-ILLUMINA


In [8]:
print(read.cigartuples)

[(4, 46), (0, 66), (4, 39)]


In [9]:
read.cigartuples = [(4, 46), (0, 66), (4, 30), (4, 9)]

In [10]:
print(read.cigartuples)

[(4, 46), (0, 66), (4, 30), (4, 9)]


In [11]:
print(read.to_string())

ST-E00130:582:HC7KJALXX:1:2204:12307:39616	129	1	9999	0	46S66M30S9S	5	18606670	0	AAGTATGAGTTGCCTAGAACAGCTCTTCCTTTATTTTCTTTTTCTGGATAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACAGAGCTAACTATTGATACAACACCTTCATGACCCTACGG	<<<<<B<<<<<<<<<<<<<<<<<<<<<<<<<<<7<<<7B<7<<<BB<7<<<'<0BB<<<<B0<<7<<0<77<B<<0'<7<<7<7BB<BB<B<B<<BB7B<BB<<BB<<B<7BB'07'<<0'BBBB<B<<<7B7B<<777B<B0<<0<<'''	SA:Z:5,18606943,-,96S55M,0,0;5,18606895,-,41M110S,0,2;	MC:Z:69M2D38M44S	MD:Z:66	NM:i:0	MQ:i:60	AS:i:66	XS:i:65	RG:Z:JKL-LU-14-cancer-wgs-ILLUMINA


# Variant

In [13]:
from handygenome import common
from handygenome.variant import varianthandler
from handygenome.vcfeditor import initvcf

in_vcf_path = '/home/users/pjh/practice/pysam/test.vcf'
out_vcf_path = '/home/users/pjh/practice/pysam/testout.vcf'
vcf = pysam.VariantFile(in_vcf_path)

In [7]:
print(vcf.header)

##fileformat=VCFv4.3
##FILTER=<ID=PASS,Description="All filters passed">
##custominfo=JULAB
##contig=<ID=1,length=100000>
##contig=<ID=2,length=100000>
##FILTER=<ID=BAR,Description="BARFILTER">
##INFO=<ID=FOO,Type=Integer,Number=1,Description="BAR">
##FORMAT=<ID=GT,Number=1,Type=String,Description="genotype">
##FORMAT=<ID=BAZ,Type=Integer,Number=1,Description="HEHE">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	s1	s2



In [12]:
vr = next(vcf.fetch())

In [11]:
print(vr)
print(vr.samples['s1']['BAZ'])
print(vr.samples['s2']['BAZ'])

1	1002	.	A	DEL	.	BAR	FOO=.	GT:BAZ	0/1:.	0/1:44

None
44


In [22]:
new_header = initvcf.create_header(chromdict=common.DEFAULT_CHROMDICTS['hg19'], samples=('s2', 's1'), vcfheader=vcf.header)

In [24]:
new_vr = varianthandler.reheader(vr, new_header)

In [26]:
print(new_vr)
print(new_vr.samples['s1']['BAZ'])
print(new_vr.samples['s2']['BAZ'])

1	1002	.	A	DEL	.	BAR	FOO=.	GT:BAZ	0/1:44	0/1:.

None
44


In [28]:
with pysam.VariantFile(out_vcf_path, mode='wb', header=new_header) as out_vcf:
    out_vcf.write(vr)
    # out_vcf.write(new_vr)

new2_vr = next(pysam.VariantFile(out_vcf_path).fetch())

print(new2_vr)
print(new2_vr.samples['s1']['BAZ'])
print(new2_vr.samples['s2']['BAZ'])

1	1002	.	A	DEL	.	BAR	FOO=.	GT:BAZ	0/1:.	0/1:44

44
None


[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/practice/pysam/testout.vcf'


In [ ]:
new_vr = 

In [86]:
new_vr = varianthandler.rename(vr, ['s2', 's1'])

In [87]:
print(vr.samples['s1']['BAZ'])
print(vr.samples['s2']['BAZ'])
print()
print(new_vr.samples['s1']['BAZ'])
print(new_vr.samples['s2']['BAZ'])

None
44

44
None


In [88]:
new_header = vcf.header.copy()
# new_header.add_sample('s3')

In [89]:
with pysam.VariantFile(out_vcf_path, mode='wb', header=new_header) as out_vcf:
    # out_vcf.write(vr)
    out_vcf.write(new_vr)

In [90]:
recovered_new_vr = next(pysam.VariantFile(out_vcf_path).fetch())

[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/practice/pysam/testout.vcf'


In [91]:
print(recovered_new_vr.samples['s1']['BAZ'])
print(recovered_new_vr.samples['s2']['BAZ'])

None
44


### allele_indices, phased

In [45]:
vr.samples.values()[-1].allele_indices = [1, 1, 0]
vr.samples.values()[-1].phased = False

In [46]:
print(vr)

1	69496	.	G	A	.	PASS	AS_SB_TABLE=20,14|4,4;ECNT=1;GERMQ=16;DP=43;AS_FilterStatus=SITE;MBQ=37,37;MFRL=364,322;MMQ=25,42;MPOS=50;NALOD=1.18;NLOD=4.21;POPAF=2.69;TLOD=20.33;context=CCG>T;repeatRegion=0;satellite=0;dbSNP_FREQ=1000Genomes:0.0009984|ExAC:0.0007382|GnomAD_exomes:0.0007058|GoESP:0.002364;dbSNP_MAX_FREQ=0.002364;dbSNP_MAX_FREQ_POP=GoESP;ponSNU_tDP=874;ponSNU_tN=30;ponSNU_vDP=0;ponSNU_vN=0;ponSNU_vafPct=0;ponPCAWG_tDP=684;ponPCAWG_tN=32;ponPCAWG_vDP=1;ponPCAWG_vN=1;ponPCAWG_vafPct=0.15;ponBGI_tDP=468;ponBGI_tN=24;ponBGI_vDP=1;ponBGI_vN=1;ponBGI_vafPct=0.21;ponPCNSL_tDP=199;ponPCNSL_tN=11;ponPCNSL_vDP=1;ponPCNSL_vN=1;ponPCNSL_vafPct=0.5	GT:AD:AF:DP:F1R2:F2R1:SB:ref_read:var_read:other_read:ref_read_lowqual:var_read_lowqual:other_read_lowqual:vaf:ref_minMQ:ref_medMQ:ref_meanMQ:ref_maxMQ:var_minMQ:var_medMQ:var_meanMQ:var_maxMQ:ref_minBQ:ref_medBQ:ref_meanBQ:ref_maxBQ:var_minBQ:var_medBQ:var_meanBQ:var_maxBQ:minLocaLt:medLocaLt:meanLocaLt:maxLocaLt:minLocaRt:medLocaRt:meanLocaRt:ma

In [6]:
print(vcf.header.samples[0])

LY12_C.Mutect2


In [27]:
vr = vcf.header.new_record()
vr.contig = '1'
vr.pos = 1000000
vr.ref = 'C'

[W::bcf_update_info] INFO/END=999999 is smaller than POS at 1:1000000


In [28]:
print(vr)

1	1000000	.	C	<NON_REF>	.	.	END=1000000	.	.	.	.	.	.	.

